<a href="https://colab.research.google.com/github/carolflyjs/cs230/blob/master/Elmo_Dot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import tensorflow as tf
import tensorflow_hub as hub
import keras
import numpy as np
from keras.layers import Input, Dense, concatenate, Dot
from keras.engine import Layer
from keras.models import Model
from keras import backend as K
from sklearn.utils import shuffle

Using TensorFlow backend.


In [0]:
def load_file(path, x_columns, y_columns, delimiter=",", train_percent = 0.8):
    df = pd.read_csv(path, delimiter=delimiter)
    df = shuffle(df)
    msk = np.random.rand(len(df)) < train_percent
    df_train = df[msk]
    df_test = df[~msk]
    X_train = df_train[x_columns]
    Y_train = df_train[y_columns]
    X_test = df_test[x_columns]
    Y_test = df_test[y_columns]
    class_dict = {
        "BT": 0,
        "NT": 1,
        "NPT": 2,
        "PT": 2,
        "RT": 3,
        "URT": 3
    }
    Y_train_int = Y_train.replace({"label": class_dict})
    Y_test_int = Y_test.replace({"label": class_dict})
    return X_train, Y_train_int, X_test, Y_test_int

In [0]:
X_train, Y_train, X_test, Y_test = load_file("./train.csv", ["source", "target"], ["label"], train_percent=0.1)
one_hot_labels = keras.utils.to_categorical(Y_train, num_classes=4)


In [0]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [0]:
def build_elmo_dot_product_model(): 
    source_input = Input(shape=(1,), dtype="string")
    source_embed = ElmoEmbeddingLayer()(source_input)
    target_input = Input(shape=(1,), dtype="string")
    target_embed = ElmoEmbeddingLayer()(target_input)
    embedding = Dot(axes=1)([source_embed, target_embed])
    pred = Dense(4, activation='softmax')(embedding)

    model = Model(inputs=[source_input, target_input], outputs=pred)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [0]:
model = build_elmo_dot_product_model()
model.fit([X_train["source"], X_train["target"]], one_hot_labels, epochs=5, steps_per_epoch=10, shuffle=True) 



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
elmo_embedding_layer_1 (ElmoEmb (None, 1024)         4           input_1[0][0]                    
__________________________________________________________________________________________________
elmo_embedding_layer_2 (ElmoEmb (None, 1024)         4           input_2[0][0]                    
____________________________________________________________________________________________

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5



10/10 [==============================] - 2035s 204s/step - loss: 13.4711 - acc: 0.1639
Epoch 2/5
10/10 [==============================] - 2046s 205s/step - loss: 13.4687 - acc: 0.1639
Epoch 3/5
10/10 [==============================] - 2044s 204s/step - loss: 13.4645 - acc: 0.1639
Epoch 4/5
10/10 [==============================] - 2052s 205s/step - loss: 13.4573 - acc: 0.1639
Epoch 5/5
 9/10 [==========================>...] - ETA: 3:24 - loss: 13.4447 - acc: 0.1639